# Investigating rank of second cumulant in CE2 

> for every zonal mode m, there is a square matrix in modes along y1,y2, whose rank is in being investigated here

In [ ]:
import numpy as np

import matplotlib as mpl
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

import matplotlib.animation as animation

In [ ]:
plt.rc('font', family='serif') ## setting for figures which I usually use in my publications
mpl.rcParams.update({'font.size': 12})
mpl.rcParams.update({'legend.labelspacing':0.25, 'legend.fontsize': 12})
mpl.rcParams.update({'errorbar.capsize': 4})

In [ ]:
def animate_data(data,dn,filename,title):

    fps = 1
    saveinterval = 5
    
    mymin = min(data[:,:,0].flatten())
    mymax = max(data[:,:,0].flatten())

    fig,ax = plt.subplots(figsize=(6,6))

    ax.set_title(title)
    im = ax.imshow((data[:,:,0]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))
    fig.colorbar(im)

    N = np.shape(data[:,:,0])[0]
    ax.set_xticks([0,N/2,N-1])
    ax.set_xticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)
    ax.set_yticks([0,N/2,N-1])
    ax.set_yticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)
    ax.set_xlabel(r'$y_1$')
    ax.set_ylabel(r'$y_2$')
    
    def animate(i):

        mymin = min(data[:,:,i].flatten())
        mymax = max(data[:,:,i].flatten())

        plt.title(title+' at t = {:0.0f} days'.format(i*saveinterval),fontsize=14)
        im.set_data(data[:,:,i])
        im.set_clim(mymin,mymax)

        return [im]

    anim = animation.FuncAnimation(fig, animate)
    anim.save(dn+filename+'.mp4', fps=fps, extra_args=['-vcodec', 'libx264'])
    
    plt.close()

In [ ]:
def animate_multi_data(data1,data2,data3,dn,filename):

    fps = 1
    saveinterval = 5
    
    mymin = min(data1[:,:,0].flatten())
    mymax = max(data1[:,:,0].flatten())

    fig,ax = plt.subplots(1,3,figsize=(12,6))

    ax[0].set_title(f'QL')
    im1 = ax[0].imshow((data1[:,:,0]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))
    ax[1].set_title(f'CE2')
    im2 = ax[1].imshow((data2[:,:,0]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))
    ax[2].set_title(f'CE2 w/ QL IC')
    im3 = ax[2].imshow((data3[:,:,0]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))

    fig.subplots_adjust(bottom=0.2)
    cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.04])
    fig.colorbar(im1, cax=cbar_ax,orientation='horizontal')

    for a in ax:
        a.set_xticks([])
        a.set_yticks([])
    
    def animate(i):

        mymin = min(data3[:,:,i].flatten())
        mymax = max(data3[:,:,i].flatten())

        plt.title('Second cumulant at t = {:0.0f} days'.format(i*saveinterval),fontsize=14)
        im1.set_data(data1[:,:,i])
        im1.set_clim(mymin,mymax)

        im2.set_data(data2[:,:,i])
        im2.set_clim(mymin,mymax)

        im3.set_data(data3[:,:,i])
#         im3.set_clim(mymin,mymax)

        return [im1,im2,im3]

    anim = animation.FuncAnimation(fig, animate)
    anim.save(dn+filename+'.mp4', fps=fps, extra_args=['-vcodec', 'libx264'])
    
    plt.close()

In [ ]:
# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """Normalise the colorbar."""
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

In [ ]:
def ranks(array):
    return len(np.where(array > 1e-10)[0])

def moderanks(array):
    nt = np.shape(array)[2]
    nm = np.shape(array)[1]
    
    R = np.reshape(np.zeros(nt*nm),(nt,nm))
    for t in np.arange(nt):
        for m in np.arange(nm):
            R[t,m] = ranks(array[:,m,t])

    return R

In [ ]:
dn = "ranktests/repeat/"

## Point Jet

> for a point jet with relaxation time of 20 days, the rank is unity for energetic zonal modes, as expected

In [ ]:
ql_pj = np.load(dn+"ql_pj_500.npz",allow_pickle=True) 
ce2_pj = np.load(dn+"ce2_pj_500.npz",allow_pickle=True) 
ce2_pj_qlic = np.load(dn+"ce2_pj_qlic_500.npz",allow_pickle=True) 

In [ ]:
M,N = 8,8

In [ ]:
Nt = len(ce2_pj['t'])
T = ce2_pj['t'][-1]
t = np.linspace(0,T,Nt)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_pj['Emt']):    
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_pj['Emt']):
    ax[1].plot(t,x,label=i,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_pj_qlic['Emt']):
    ax[2].plot(t,x,label=i,c=colors[i])


for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.0,0.75),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_pj_500.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,5))

colors = pl.cm.seismic(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_pj['Em1nt']):    
    ax[0].plot(ql_pj['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_pj['Em1nt']):
    ax[1].plot(ce2_pj['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_pj_qlic['Em1nt']):
    ax[2].plot(ce2_pj_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-6,1e0)
#     a.set_ylim(0,3.5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.35,0.95),ncol=1)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,5))

colors = pl.cm.seismic(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_pj['Em2nt']):    
    ax[0].plot(ql_pj['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_pj['Em2nt']):
    ax[1].plot(ce2_pj['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_pj_qlic['Em2nt']):
    ax[2].plot(ce2_pj_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-6,1e-1)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.35,0.95),ncol=1)

plt.show()

In [ ]:
R_ce2_pj = moderanks(ce2_pj['mEVs'])
R_ce2_pj_qlic = moderanks(ce2_pj_qlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(6):
    ax[0].plot(t,R_ce2_pj[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(6):
    ax[1].plot(t,R_ce2_pj_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_pj['mEVs'][:,0,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_pj_qlic['mEVs'][:,0,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
#     a.set_yscale('log')
#     a.set_ylim(1e-16,1e3)
    a.set_ylim(0,5)
    a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 2 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_pj['mEVs'][:,1,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 2 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_pj_qlic['mEVs'][:,1,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
#     a.set_yscale('log')
    a.set_ylim(0,0.5)
    a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 3 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_pj['mEVs'][:,2,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 3 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_pj_qlic['mEVs'][:,2,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
#     a.set_yscale('log')
    a.set_ylim(0,1e-9)
    a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m3.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(12,6))

index = 8

mymin = min(ql_pj['Fyym1'][:,:,index].flatten())
mymax = max(ql_pj['Fyym1'][:,:,index].flatten())

ax[0].set_title(f'QL')
im = ax[0].imshow((ql_pj['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))
ax[1].set_title(f'CE2')
im = ax[1].imshow((ce2_pj['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))
ax[2].set_title(f'CE2 w/ QL IC')
im = ax[2].imshow((ce2_pj_qlic['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

fig.subplots_adjust(bottom=0.2)
cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.04])
fig.colorbar(im, cax=cbar_ax,orientation='horizontal')

plt.title('Second cumulant for m = 1 at t = {:0.0f} days'.format(ql_pj['t'][index]),fontsize=14)

for a in ax:
    a.set_ylabel(r'$y_2$',fontsize=14)
    a.set_xlabel(r'$y_1$',fontsize=14)
    a.set_xticks([])
    a.set_yticks([])

plt.show()

In [ ]:
# animate_multi_data(ql_pj['Fyym1'],ce2_pj['Fyym1'],ce2_pj_qlic['Fyym1'],dn,'pj_ql_ce2_fyym1')

In [ ]:
animate_data(ql_pj['Fyym1'],dn,'pj_ql_fyym1','QL')
animate_data(ce2_pj['Fyym1'],dn,'pj_ce2_fyym1','CE2')
animate_data(ce2_pj_qlic['Fyym1'],dn,'pj_ce2_qlic_fyym1','CE2 w/ QL IC')

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(12,6))

index = -1

mymin = min(ql_pj['Fyym2'][:,:,index].flatten())
mymax = max(ql_pj['Fyym2'][:,:,index].flatten())

ax[0].set_title(f'QL')
im = ax[0].imshow((ql_pj['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))
ax[1].set_title(f'CE2')
im = ax[1].imshow((ce2_pj['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))
ax[2].set_title(f'CE2 w/ QL IC')
im = ax[2].imshow((ce2_pj_qlic['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

fig.subplots_adjust(bottom=0.2)
cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.04])
fig.colorbar(im, cax=cbar_ax,orientation='horizontal')

plt.title('Second cumulant for m = 2 at t = {:0.0f} days'.format(ql_pj['t'][index]),fontsize=14)

for a in ax:
    a.set_ylabel(r'$y_2$',fontsize=14)
    a.set_xlabel(r'$y_1$',fontsize=14)
    a.set_xticks([])
    a.set_yticks([])

plt.show()

In [ ]:
# animate_multi_data(ql_pj['Fyym2'],ce2_pj['Fyym2'],ce2_pj_qlic['Fyym2'],dn,'ql_ce2_fyym2')
animate_data(ql_pj['Fyym2'],dn,'pj_ql_fyym2','QL')
animate_data(ce2_pj['Fyym2'],dn,'pj_ce2_fyym2','CE2')
animate_data(ce2_pj_qlic['Fyym2'],dn,'pj_ce2_qlic_fyym2','CE2 w/ QL IC')

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,6))

im = ax[0].imshow((ql_pj['Uxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic")
im = ax[1].imshow((ce2_pj['Uxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic")
im = ax[2].imshow((ce2_pj_qlic['Uxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic")

for a in ax:
    a.set_xticks([0,M,2*M-2])
    a.set_xticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)
    a.set_yticks([0,M,2*M-2])
    a.set_yticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,6))

im = ax[0].imshow((ql_pj['Vxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic")
im = ax[1].imshow((ce2_pj['Vxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic")
im = ax[2].imshow((ce2_pj_qlic['Vxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic")

for a in ax:
    a.set_xticks([0,M,2*M-2])
    a.set_xticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)
    a.set_yticks([0,M,2*M-2])
    a.set_yticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)

plt.show()

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(15,6))

im = ax[0].imshow((ql_pj['Vyt']),cmap="RdBu_r",origin="lower",interpolation="bicubic")
im = ax[1].imshow((ce2_pj['Vyt']),cmap="RdBu_r",origin="lower",interpolation="bicubic")
im = ax[2].imshow((ce2_pj_qlic['Vyt']),cmap="RdBu_r",origin="lower",interpolation="bicubic")

for a in ax:
#     a.set_xticks([0,M,2*M-2])
#     a.set_xticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)
    a.set_yticks([0,M,2*M-2])
    a.set_yticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)

plt.show()

## Kolmogorov Flow

> here is where disagreements occur between QL and CE2

### 8x8 grid

In [ ]:
dn = "ranktests/repeat/kolmogorov/8x8/"

In [ ]:
ql_kf = np.load(dn+"ql_kf_500.npz",allow_pickle=True) 
ce2_kf = np.load(dn+"ce2_kf_500.npz",allow_pickle=True) 
ce2_kf_qlic = np.load(dn+"ce2_kf_qlic_500.npz",allow_pickle=True) 

In [ ]:
M,N = 8,8

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'QL')

t = ql_kf['t']
for i,x in enumerate(ql_kf['Emt']):    
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Emt']):
    ax[1].plot(x,label=i,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Emt']):
    ax[2].plot(x,label=i,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e4)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.0,0.75),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_kf_500.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,5))

colors = pl.cm.jet(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Em1nt']):    
    ax[0].plot(ql_kf['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Em1nt']):
    ax[1].plot(ce2_kf['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Em1nt']):
    ax[2].plot(ce2_kf_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-6,1e3)
#     a.set_ylim(0,3.5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.35,0.95),ncol=1)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(14,5))

colors = pl.cm.jet(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Em2nt']):    
    ax[0].plot(ql_kf['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Em2nt']):
    ax[1].plot(ce2_kf['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Em2nt']):
    ax[2].plot(ce2_kf_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e2)
#     a.set_ylim(0,3.5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.35,0.95),ncol=1)

plt.show()

In [ ]:
animate_data(ql_kf['Vxy'],dn,'kf_ql_8x8_vxy','QL vorticity')
animate_data(ce2_kf['Vxy'],dn,'kf_ce2_8x8_vxy','CE2 vorticity')
animate_data(ce2_kf_qlic['Vxy'],dn,'kf_ce2_qlic_8x8_vxy','CE2 w/ QL IC vorticity')

In [ ]:
animate_data(ql_kf['Fyym1'],dn,'kf_ql_8x8_fyym1','QL')
animate_data(ce2_kf['Fyym1'],dn,'kf_ce2_8x8_fyym1','CE2')
animate_data(ce2_kf_qlic['Fyym1'],dn,'kf_ce2_8x8_qlic_fyym1','CE2 w/ QL IC')

In [ ]:
animate_data(ql_kf['Fyym2'],dn,'kf_ql_8x8_fyym2','QL')
animate_data(ce2_kf['Fyym2'],dn,'kf_ce2_8x8_fyym2','CE2')
animate_data(ce2_kf_qlic['Fyym2'],dn,'kf_ce2_8x8_qlic_fyym2','CE2 w/ QL IC')

In [ ]:
# fig,ax = plt.subplots(1,3,figsize=(12,6))

# index = -1

# ax[0].set_title(f'QL')
# mymin = min(ql_kf['Fyym1'][:,:,index].flatten())
# mymax = max(ql_kf['Fyym1'][:,:,index].flatten())
# im1 = ax[0].imshow((ql_kf['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))

# ax[1].set_title(f'CE2')
# mymin = min(ce2_kf['Fyym1'][:,:,index].flatten())
# mymax = max(ce2_kf['Fyym1'][:,:,index].flatten())
# im2 = ax[1].imshow((ce2_kf['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

# ax[2].set_title(f'CE2 w/ QL IC')
# mymin = min(ce2_kf_qlic['Fyym1'][:,:,index].flatten())
# mymax = max(ce2_kf_qlic['Fyym1'][:,:,index].flatten())
# im3 = ax[2].imshow((ce2_kf_qlic['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

# fig.subplots_adjust(bottom=0.2)
# cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.04])
# fig.colorbar(im3, cax=cbar_ax,orientation='horizontal')

# plt.title('Second cumulant for m = 1 at t = {:0.0f} days'.format(ql_kf['t'][index]),fontsize=14)

# for a in ax:
#     a.set_ylabel(r'$y_2$',fontsize=14)
#     a.set_xlabel(r'$y_1$',fontsize=14)
#     a.set_xticks([])
#     a.set_yticks([])

# plt.show()

In [ ]:
# fig,ax = plt.subplots(1,3,figsize=(12,6))

# index = -1

# ax[0].set_title(f'QL')
# mymin = min(ql_kf['Fyym2'][:,:,index].flatten())
# mymax = max(ql_kf['Fyym2'][:,:,index].flatten())
# im1 = ax[0].imshow((ql_kf['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))

# ax[1].set_title(f'CE2')
# mymin = min(ce2_kf['Fyym2'][:,:,index].flatten())
# mymax = max(ce2_kf['Fyym2'][:,:,index].flatten())
# im2 = ax[1].imshow((ce2_kf['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

# ax[2].set_title(f'CE2 w/ QL IC')
# mymin = min(ce2_kf_qlic['Fyym2'][:,:,index].flatten())
# mymax = max(ce2_kf_qlic['Fyym2'][:,:,index].flatten())
# im3 = ax[2].imshow((ce2_kf_qlic['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

# fig.subplots_adjust(bottom=0.2)
# cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.04])
# fig.colorbar(im1, cax=cbar_ax,orientation='horizontal')

# plt.title('Second cumulant for m = 2 at t = {:0.0f} days'.format(ql_kf['t'][index]),fontsize=14)

# for a in ax:
#     a.set_ylabel(r'$y_2$',fontsize=14)
#     a.set_xlabel(r'$y_1$',fontsize=14)
#     a.set_xticks([])
#     a.set_yticks([])

# plt.show()

In [ ]:
R_ce2_kf = moderanks(ce2_kf['mEVs'])
R_ce2_kf_qlic = moderanks(ce2_kf_qlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(M-1):
    ax[0].plot(R_ce2_kf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(M-1):
    ax[1].plot(R_ce2_kf_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
t = ce2_kf['t']
for i,x in enumerate(ce2_kf['mEVs'][:,0,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,0,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
#     a.set_ylim(0,400)
    a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 2 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,1,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 2 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,1,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
#     a.set_yscale('log')
#     a.set_ylim(1e-16,1e3)
    a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,6))

im = ax[0].imshow((ql_kf['Vxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic",vmin=-30,vmax=30)
im = ax[1].imshow((ce2_kf['Vxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic",vmin=-30,vmax=30)
im = ax[2].imshow((ce2_kf_qlic['Vxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic",vmin=-30,vmax=30)

for a in ax:
    a.set_xticks([0,M,2*M-2])
    a.set_xticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)
    a.set_yticks([0,M,2*M-2])
    a.set_yticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,6))

im = ax[0].imshow((ql_kf['Emn'][:,:,-1]),cmap="nipy_spectral_r",origin="lower",interpolation="bicubic")
im = ax[1].imshow((ce2_kf['Emn'][:,:,-1]),cmap="nipy_spectral_r",origin="lower",interpolation="bicubic")
im = ax[2].imshow((ce2_kf_qlic['Emn'][:,:,-1]),cmap="nipy_spectral_r",origin="lower",interpolation="bicubic")

for a in ax:
    a.set_xticks([0,M,2*M-2])
    a.set_xticklabels([r'$-M$',r'$0$',r'$M$'],fontsize=14)
    a.set_yticks([0,M,2*M-2])
    a.set_yticklabels([r'$-M$',r'$0$',r'$M$'],fontsize=14)

plt.show()

## 11x11 grid

In [ ]:
dn = "ranktests/repeat/kolmogorov/12x12/"

In [ ]:
ql_kf = np.load(dn+"ql_kf_11x11_1000.npz",allow_pickle=True) 
ce2_kf = np.load(dn+"ce2_kf_11x11_500.npz",allow_pickle=True) 
ce2_kf_qlic = np.load(dn+"ce2_kf_11x11_qlic_500.npz",allow_pickle=True) 

In [ ]:
M,N = 11,11

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'QL')
t = ql_kf['t']
for i,x in enumerate(ql_kf['Emt']):    
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
t = ce2_kf['t']
for i,x in enumerate(ce2_kf['Emt']):
    ax[1].plot(t,x,label=i,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
t = ce2_kf_qlic['t']
for i,x in enumerate(ce2_kf_qlic['Emt']):
    ax[2].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.0,0.95),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_kf_500.png',bbox_inches='tight')

In [ ]:
animate_data(ql_kf['Vxy'],dn,'kf_ql_11x11_vxy','QL vorticity')
animate_data(ce2_kf['Vxy'],dn,'kf_ce2_11x11_vxy','CE2 vorticity')
animate_data(ce2_kf_qlic['Vxy'],dn,'kf_ce2_qlic_11x11_vxy','CE2 w/ QL IC vorticity')

In [ ]:
animate_data(ql_kf['Fyym1'],dn,'kf_ql_11x11_fyym1','QL c2 m = 1')
animate_data(ce2_kf['Fyym1'],dn,'kf_ce2_11x11_fyym1','CE2 c2 m = 1')
animate_data(ce2_kf_qlic['Fyym1'],dn,'kf_ce2_11x11_qlic_fyym1','CE2 w/ QL IC c2 m = 1')

In [ ]:
animate_data(ql_kf['Fyym2'],dn,'kf_ql_11x11_fyym2','QL c2 m = 2')
animate_data(ce2_kf['Fyym2'],dn,'kf_ce2_11x11_fyym2','CE2 c2 m = 2')
animate_data(ce2_kf_qlic['Fyym2'],dn,'kf_ce2_11x11_qlic_fyym2','CE2 w/ QL IC c2 m = 2')

In [ ]:
animate_data(ql_kf['Fyym3'],dn,'kf_ql_11x11_fyym3','QL c2 m = 3')
animate_data(ce2_kf['Fyym3'],dn,'kf_ce2_11x11_fyym3','CE2 c2 m = 3')
animate_data(ce2_kf_qlic['Fyym3'],dn,'kf_ce2_11x11_qlic_fyym3','CE2 w/ QL IC c2 m = 3')

In [ ]:
R_ce2_kf = moderanks(ce2_kf['mEVs'])
R_ce2_kf_qlic = moderanks(ce2_kf_qlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(M-1):
    ax[0].plot(R_ce2_kf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(M-1):
    ax[1].plot(R_ce2_kf_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
t = ce2_kf['t']
for i,x in enumerate(ce2_kf['mEVs'][:,0,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,0,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
#     a.set_ylim(0,400)
    a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 2 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,1,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 2 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,1,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
#     a.set_ylim(0,10)
    a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

### 16x16 grid

In [ ]:
dn = "ranktests/repeat/kolmogorov/16x16/"

In [ ]:
ql_kf = np.load(dn+"ql_kf_16x16_1000.npz",allow_pickle=True) 
ce2_kf = np.load(dn+"ce2_kf_16x16_500.npz",allow_pickle=True)
ce2_kf_qlic = np.load(dn+"ce2_kf_16x16_qlic_500.npz",allow_pickle=True)

In [ ]:
M,N=16,16

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'QL')
t = ql_kf['t']
for i,x in enumerate(ql_kf['Emt']):    
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
t = ce2_kf['t']
for i,x in enumerate(ce2_kf['Emt']):
    ax[1].plot(t,x,label=i,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
t = ce2_kf_qlic['t']
for i,x in enumerate(ce2_kf_qlic['Emt']):
    ax[2].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.0,0.95),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_kf_500.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.jet(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Em0nt']):    
    ax[0].plot(ql_kf['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Em0nt']):
    ax[1].plot(ce2_kf['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Em0nt']):
    ax[2].plot(ce2_kf_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e5)
#     a.set_ylim(0,3.5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(0.45,-0.15),ncol=9)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.jet(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Em1nt']):    
    ax[0].plot(ql_kf['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Em1nt']):
    ax[1].plot(ce2_kf['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Em1nt']):
    ax[2].plot(ce2_kf_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e3)
#     a.set_ylim(0,3.5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(0.45,-0.15),ncol=9)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.jet(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Em2nt']):    
    ax[0].plot(ql_kf['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Em2nt']):
    ax[1].plot(ce2_kf['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Em2nt']):
    ax[2].plot(ce2_kf_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-20,1e1)
#     a.set_ylim(0,3.5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(0.45,-0.15),ncol=9)

plt.show()

In [ ]:
R_ce2_kf = moderanks(ce2_kf['mEVs'])
R_ce2_kf_qlic = moderanks(ce2_kf_qlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
t = ce2_kf['t']
for i in np.arange(M-1):
    ax[0].plot(t,R_ce2_kf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
t = ce2_kf_qlic['t']
for i in np.arange(M-1):
    ax[1].plot(t,R_ce2_kf_qlic[:,i],label=i+1,c=colors[i+1])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

ax[1].legend(bbox_to_anchor=(1.05,1.25),ncol=1)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*M-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
t = ce2_kf['t']
for i,x in enumerate(ce2_kf['mEVs'][:,0,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
t = ce2_kf_qlic['t']
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,0,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
#     a.set_yscale('log')
#     a.set_ylim(0,0)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*16-1))

ax[0].set_title(f'm = 2 spectrum for CE2 w/ rand IC')
t = ce2_kf['t']
for i,x in enumerate(ce2_kf['mEVs'][:,1,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 2 spectrum for CE2 w/ QL IC')
t = ce2_kf_qlic['t']
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,1,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
#     a.set_yscale('log')
#     a.set_ylim(1e-16,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.jet(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Em0nt']):    
    ax[0].plot(ql_kf['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Em0nt']):
    ax[1].plot(ce2_kf['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Em0nt']):
    ax[2].plot(ce2_kf_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e5)
#     a.set_ylim(0,3.5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(0.45,-0.15),ncol=9)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.jet(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Em1nt']):    
    ax[0].plot(ql_kf['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Em1nt']):
    ax[1].plot(ce2_kf['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Em1nt']):
    ax[2].plot(ce2_kf_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e3)
#     a.set_ylim(0,3.5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(0.45,-0.15),ncol=9)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.jet(np.linspace(0,1,2*N-1))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Em2nt']):    
    ax[0].plot(ql_kf['t'],x,label=i-N+1,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Em2nt']):
    ax[1].plot(ce2_kf['t'],x,label=i-N+1,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Em2nt']):
    ax[2].plot(ce2_kf_qlic['t'],x,label=i-N+1,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-20,1e1)
#     a.set_ylim(0,3.5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(0.45,-0.15),ncol=9)

plt.show()

In [ ]:
animate_data(ql_kf['Fyym1'],dn,'kf_ql_16x16_fyym1','QL c2 for m = 1')
animate_data(ce2_kf['Fyym1'],dn,'kf_ce2_16x16_fyym1','CE2 c2 for m = 1')
animate_data(ce2_kf_qlic['Fyym1'],dn,'kf_ce2_qlic_16x16_fyym1','CE2 QL IC c2 for m = 1')

In [ ]:
animate_data(ql_kf['Fyym2'],dn,'kf_ql_16x16_fyym2','QL c2 for m = 2')
animate_data(ce2_kf['Fyym2'],dn,'kf_ce2_16x16_fyym2','CE2 c2 for m = 2')
animate_data(ce2_kf_qlic['Fyym2'],dn,'kf_ce2_qlic_16x16_fyym2','CE2 QL IC c2 for m = 2')

In [ ]:
animate_data(ql_kf['Vxy'],dn,'kf_ql_vxy','QL vorticity')
animate_data(ce2_kf['Vxy'],dn,'kf_ce2_vxy','CE2 vorticity')
animate_data(ce2_kf_qlic['Vxy'],dn,'kf_ce2_qlic_vxy','CE2 w/ QL IC vorticity')

In [ ]:
# fig,ax = plt.subplots(1,3,figsize=(12,6))

# index = 42

# ax[0].set_title(f'QL')
# mymin = min(ql_kf['Fyym1'][:,:,index].flatten())
# mymax = max(ql_kf['Fyym1'][:,:,index].flatten())
# im1 = ax[0].imshow((ql_kf['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))

# ax[1].set_title(f'CE2')
# mymin = min(ce2_kf['Fyym1'][:,:,index].flatten())
# mymax = max(ce2_kf['Fyym1'][:,:,index].flatten())
# im2 = ax[1].imshow((ce2_kf['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

# ax[2].set_title(f'CE2 w/ QL IC')
# mymin = min(ce2_kf_qlic['Fyym1'][:,:,index].flatten())
# mymax = max(ce2_kf_qlic['Fyym1'][:,:,index].flatten())
# im3 = ax[2].imshow((ce2_kf_qlic['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

# fig.subplots_adjust(bottom=0.2)
# cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.04])
# fig.colorbar(im3, cax=cbar_ax,orientation='horizontal')

# plt.title('Second cumulant for m = 1 at t = {:0.0f} days'.format(ql_kf['t'][index]),fontsize=14)

# for a in ax:
#     a.set_ylabel(r'$y_2$',fontsize=14)
#     a.set_xlabel(r'$y_1$',fontsize=14)
#     a.set_xticks([])
#     a.set_yticks([])

# plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

ax[0].set_title(f'QL')
t = ql_kf['t']
# for i,x in enumerate(ql_kf['Emt']):    
ax[0].scatter(ql_kf['Emt'][1],ql_kf['Emt'][2],c=ql_kf['t'],cmap=pl.cm.cool)
ax[0].plot(ql_kf['Emt'][1],ql_kf['Emt'][2],'-',alpha=0.25)

ax[1].set_title(f'CE2')
t = ce2_kf['t']
ax[1].scatter(ce2_kf['Emt'][1],ce2_kf['Emt'][2],c=ce2_kf['t'],cmap=pl.cm.cool)
ax[1].plot(ce2_kf['Emt'][1],ce2_kf['Emt'][2],'-',alpha=0.25)


ax[2].set_title(f'CE2 w/ QL IC')
t = ce2_kf_qlic['t']
ax[2].scatter(ce2_kf_qlic['Emt'][1],ce2_kf_qlic['Emt'][2],c=ce2_kf_qlic['t'],cmap=pl.cm.cool)
ax[2].plot(ce2_kf_qlic['Emt'][1],ce2_kf_qlic['Emt'][2],'-',alpha=0.25)

for a in ax:
    a.set_xlabel(r'$m = 1$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e5)

ax[0].set_ylabel(r'$m=2$',fontsize=14)     
# ax[2].legend(bbox_to_anchor=(1.0,0.95),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_kf_500.png',bbox_inches='tight')

In [ ]:
# fig,ax = plt.subplots(1,3,figsize=(12,6))

# index = 40

# ax[0].set_title(f'QL')
# mymin = min(ql_kf['Fyym2'][:,:,index].flatten())
# mymax = max(ql_kf['Fyym2'][:,:,index].flatten())
# im1 = ax[0].imshow((ql_kf['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))

# ax[1].set_title(f'CE2')
# mymin = min(ce2_kf['Fyym2'][:,:,index].flatten())
# mymax = max(ce2_kf['Fyym2'][:,:,index].flatten())
# im2 = ax[1].imshow((ce2_kf['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

# ax[2].set_title(f'CE2 w/ QL IC')
# mymin = min(ce2_kf_qlic['Fyym2'][:,:,index].flatten())
# mymax = max(ce2_kf_qlic['Fyym2'][:,:,index].flatten())
# im3 = ax[2].imshow((ce2_kf_qlic['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

# fig.subplots_adjust(bottom=0.2)
# cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.04])
# fig.colorbar(im3, cax=cbar_ax,orientation='horizontal')

# plt.title('Second cumulant for m = 1 at t = {:0.0f} days'.format(ql_kf['t'][index]),fontsize=14)

# for a in ax:
#     a.set_ylabel(r'$y_2$',fontsize=14)
#     a.set_xlabel(r'$y_1$',fontsize=14)
#     a.set_xticks([])
#     a.set_yticks([])

# plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,6))

im = ax[0].imshow((ql_kf['Vxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic",vmin=-30,vmax=30)
im = ax[1].imshow((ce2_kf['Vxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic",vmin=-30,vmax=30)
im = ax[2].imshow((ce2_kf_qlic['Vxy'][:,:,-1]),cmap="RdBu_r",origin="lower",interpolation="bicubic",vmin=-30,vmax=30)

for a in ax:
    a.set_xticks([0,M,2*M-2])
    a.set_xticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)
    a.set_yticks([0,M,2*M-2])
    a.set_yticklabels([r'$-\pi$',r'$0$',r'$\pi$'],fontsize=14)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,6))

im = ax[0].imshow((ql_kf['Emn'][:,:,-1]),cmap="nipy_spectral_r",origin="lower",interpolation="bicubic")
im = ax[1].imshow((ce2_kf['Emn'][:,:,-1]),cmap="nipy_spectral_r",origin="lower",interpolation="bicubic")
im = ax[2].imshow((ce2_kf_qlic['Emn'][:,:,-1]),cmap="nipy_spectral_r",origin="lower",interpolation="bicubic")

for a in ax:
    a.set_xticks([0,16,2*16-2])
    a.set_xticklabels([r'$-M$',r'$0$',r'$M$'],fontsize=14)
    a.set_yticks([0,16,2*16-2])
    a.set_yticklabels([r'$-M$',r'$0$',r'$M$'],fontsize=14)

plt.show()

# 24x24 grid

In [ ]:
ql_kf = np.load(dn+"ql_kf_24x24_500.npz",allow_pickle=True) 
# ce2_kf = np.load(dn+"ce2_kf_16x16_500.npz",allow_pickle=True) # RK4 -> DP5 explodes
# ce2_kf_qlic = np.load(dn+"ce2_kf_16x16_qlic_500.npz",allow_pickle=True)  # DP5 

In [ ]:
M,N = 24,24

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'QL')
t = ql_kf['t']
for i,x in enumerate(ql_kf['Emt']):    
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
t = ce2_kf['t']
# for i,x in enumerate(ce2_kf['Emt']):
#     ax[1].plot(t,x,label=i,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
t = ce2_kf_qlic['t']
# for i,x in enumerate(ce2_kf_qlic['Emt']):
#     ax[2].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.0,0.95),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_kf_500.png',bbox_inches='tight')

## Low viscosity solution

In [ ]:
ql_kf = np.load(dn+"ql_kf_nu01_500.npz",allow_pickle=True) 
ce2_kf = np.load(dn+"ce2_kf_nu01_500.npz",allow_pickle=True) 
ce2_kf_qlic = np.load(dn+"ce2_kf_nu01_qlic_500.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.nipy_spectral(np.linspace(0,1,9))

ax[0].set_title(f'QL')
t = ql_kf['t']
for i,x in enumerate(ql_kf['Emt'].T):    
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
t = ce2_kf['t']
for i,x in enumerate(ce2_kf['Emtav'].T):
    ax[1].plot(t,x,label=i,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
t = ce2_kf_qlic['t']
for i,x in enumerate(ce2_kf_qlic['Emtav'].T):
    ax[2].plot(t,x,label=i,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.0,0.75),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_kf_500.png',bbox_inches='tight')

In [ ]:
R_ce2_kf = moderanks(ce2_kf['mEVs'])
R_ce2_kf_qlic = moderanks(ce2_kf_qlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(6):
    ax[0].plot(R_ce2_kf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(6):
    ax[1].plot(R_ce2_kf_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*11-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,0,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,0,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
ql_kf = np.load(dn+"ql_kf_11x11_500.npz",allow_pickle=True) 
ce2_kf = np.load(dn+"ce2_kf_11x11_500.npz",allow_pickle=True) # RK4 -> DP5 explodes
ce2_kf_qlic = np.load(dn+"ce2_kf_11x11_qlic_500.npz",allow_pickle=True)  # DP5 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.nipy_spectral(np.linspace(0,1,11))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Emt'].T):    
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Emt'].T):
    ax[1].plot(x,label=i,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Emt'].T):
    ax[2].plot(x,label=i,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.0,0.75),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_kf_500.png',bbox_inches='tight')

In [ ]:
R_ce2_kf = moderanks(ce2_kf['mEVs'])
R_ce2_kf_qlic = moderanks(ce2_kf_qlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*11-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,0,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,0,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(6):
    ax[0].plot(R_ce2_kf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(6):
    ax[1].plot(R_ce2_kf_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*11-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,0,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,0,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

### positivity condition applied

> with non-positive eigenvalues removed, the answer is the same

In [ ]:
ql_kf = np.load(dn+"ql_kf_500.npz",allow_pickle=True) 
ce2_kf = np.load(dn+"ce2_kf_poscheck_500.npz",allow_pickle=True) 
ce2_kf_qlic = np.load(dn+"ce2_kf_poscheck_qlic_500.npz",allow_pickle=True) 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'QL')
for i,x in enumerate(ql_kf['Emt'].T):    
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Emtav'].T):
    ax[1].plot(x,label=i,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Emtav'].T):
    ax[2].plot(x,label=i,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.0,0.75),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_kf_500.png',bbox_inches='tight')

In [ ]:
R_ce2_kf = moderanks(ce2_kf['mEVs'])
R_ce2_kf_qlic = moderanks(ce2_kf_qlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(6):
    ax[0].plot(R_ce2_kf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(6):
    ax[1].plot(R_ce2_kf_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,0,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,0,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
#     a.set_yscale('log')
    a.set_ylim(-1e-6,1e-6)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 2 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,1,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 2 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,1,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
## t = 50 days

In [ ]:
ce2_kf = np.load(dn+"ce2_kf_50.npz",allow_pickle=True) 
ce2_kf_qlic = np.load(dn+"ce2_kf_qlic_50.npz",allow_pickle=True) 

In [ ]:
R_ce2_kf = moderanks(ce2_kf['mEVs'])
R_ce2_kf_qlic = moderanks(ce2_kf_qlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(6):
    ax[0].plot(R_ce2_kf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(6):
    ax[1].plot(R_ce2_kf_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,0,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,0,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 2 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,1,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 2 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,1,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

### 9x9 grid

In [ ]:
ql_kf = np.load(dn+"ql_kf_9x9_500.npz",allow_pickle=True) 
ce2_kf = np.load(dn+"ce2_kf_9x9_500.npz",allow_pickle=True) # RK4 -> DP5 explodes
ce2_kf_qlic = np.load(dn+"ce2_kf_9x9_qlic_500.npz",allow_pickle=True)  # DP5 

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,5))

M,N = 9,9
colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'QL')
# ax[0].plot(ql_kf['Emt'].T[2],c='k')

for i,x in enumerate(ql_kf['Emt']):    
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_kf['Emt']):
    ax[1].plot(x,label=i,c=colors[i])

ax[2].set_title(f'CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['Emt']):
    ax[2].plot(x,label=i,c=colors[i])

for a in ax:
    
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e5)

ax[0].set_ylabel(r'$E(m)$',fontsize=14)
ax[2].legend(bbox_to_anchor=(1.0,0.75),ncol=1)

plt.show()
# plt.savefig(dn+'figures/qlce2_kf_500.png',bbox_inches='tight')

In [ ]:
R_ce2_kf = moderanks(ce2_kf['mEVs'])
R_ce2_kf_qlic = moderanks(ce2_kf_qlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(6):
    ax[0].plot(R_ce2_kf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(6):
    ax[1].plot(R_ce2_kf_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*10-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,0,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,0,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-9,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*10-1))

ax[0].set_title(f'm = 2 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,1,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 2 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,1,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*10-1))

ax[0].set_title(f'm = 3 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_kf['mEVs'][:,2,:]):
    ax[0].plot(x,label=i,c=colors[i])

ax[1].set_title(f'm = 3 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_kf_qlic['mEVs'][:,2,:]):
    ax[1].plot(x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-16,1e3)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

## Stochastic jet

> for a stochastically forced setting, with modes 4,5,6 stochastically forced, QL retains energy only in these modes (and in the mean of course)
> the rank of second cumulant in CE2 is non-unity for these modes.

In [ ]:
ql_sf = np.load(dn+"ql_sf_500.npz",allow_pickle=True) 
ce2_sf = np.load(dn+"ce2_sf_500.npz",allow_pickle=True) 
ce2_sf_qlic = np.load(dn+"ce2_sf_qlic_500.npz",allow_pickle=True) 
# ce2_sf_kfqlic = np.load(dn+"ce2_sf_kfqlic_500.npz",allow_pickle=True) 

In [ ]:
M,N = 8,8

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(15,8))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0,0].set_title(f'QL')
for i,x in enumerate(ql_sf['Emt']):    
    ax[0,0].plot(t,x,label=i,c=colors[i])

ax[0,1].set_title(f'CE2')
for i,x in enumerate(ce2_sf['Emt']):
    ax[0,1].plot(t,x,label=i,c=colors[i])

ax[1,0].set_title(f'CE2 w/ Stochastic QL IC')
for i,x in enumerate(ce2_sf_qlic['Emt']):
    ax[1,0].plot(t,x,label=i,c=colors[i])

ax[1,1].set_title(f'CE2 w/ Kolmogorov flow QL IC')
# for i,x in enumerate(ce2_sf_kfqlic['Emt']):
#     ax[1,1].plot(t,x,label=i,c=colors[i])

ax[0,0].set_ylabel(r'$E(m)$',fontsize=14)
ax[1,0].set_ylabel(r'$E(m)$',fontsize=14)

ax[0,1].legend(bbox_to_anchor=(1.0,0.75),ncol=1)

for a in ax.flatten():
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-6,1e1)

plt.show()
# plt.savefig(dn+'figures/qlce2_sf_500.png',bbox_inches='tight')

In [ ]:
R_ce2_sf = moderanks(ce2_sf['mEVs'])
R_ce2_sf_qlic = moderanks(ce2_sf_qlic['mEVs'])
# R_ce2_sf_kfqlic = moderanks(ce2_sf_kfqlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(6):
    ax[0].plot(t,R_ce2_sf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(6):
    ax[1].plot(t,R_ce2_sf_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 1 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_sf['mEVs'][:,0,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 1 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_sf_qlic['mEVs'][:,0,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
#     a.set_ylim(1e-9,1e3)
    a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral_r(np.linspace(0,1,2*N-1))

ax[0].set_title(f'm = 5 spectrum for CE2 w/ rand IC')
for i,x in enumerate(ce2_sf['mEVs'][:,4,:]):
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'm = 5 spectrum for CE2 w/ QL IC')
for i,x in enumerate(ce2_sf_qlic['mEVs'][:,4,:]):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_yscale('log')
    a.set_ylim(1e-3,1e0)
    a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/spectrum_ce2_pj_m1.png',bbox_inches='tight')

In [ ]:
dn = "ranktests/repeat/"

In [ ]:
ql_sf = np.load(dn+"ql_sf_test_500.npz",allow_pickle=True) 
ce2_sf = np.load(dn+"ce2_sf_test_500.npz",allow_pickle=True) 

In [ ]:
M,N = 12,12

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,8))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'QL')
t = ql_sf['t']
for i,x in enumerate(ql_sf['Emt']):    
    ax[0].plot(t,x,label=i,c=colors[i])

ax[1].set_title(f'CE2')
for i,x in enumerate(ce2_sf['Emt']):
    ax[1].plot(t,x,label=i,c=colors[i])

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
#     a.set_yscale('log')
    a.set_ylim(1e-6,0.2)
#     a.set_xlim(0,T)
    a.set_ylabel(r'$Eigenvalues$',fontsize=14)

plt.show()



In [ ]:
ql_sf = np.load(dn+"ql_sf_test_1000.npz",allow_pickle=True) 
ce2_sf = np.load(dn+"ce2_sf_test_1000.npz",allow_pickle=True) 
ce2_sf_qlic = np.load(dn+"ce2_sf_qlic_test2_1000.npz",allow_pickle=True)

In [ ]:
fig,ax = plt.subplots(figsize=(15,5))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax.set_title(f'QL')
t = ql_sf['t']
for i,x in enumerate(ql_sf['Emtav']):    
    ax.plot(t,x,label=i,c=colors[i])

# ax.set_title(f'CE2')
# t = ce2_sf['t']
# for i,x in enumerate(ce2_sf['Emt']):
#     ax.plot(t,x,label=i,c=colors[i])

ax.set_yscale('log')
ax.set_ylim(1e-16,1e0)

ax.legend()
plt.show()


In [ ]:
R_ce2_sf = moderanks(ce2_sf['mEVs'])
R_ce2_sf_qlic = moderanks(ce2_sf_qlic['mEVs'])
# R_ce2_sf_kfqlic = moderanks(ce2_sf_kfqlic['mEVs'])

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,3))

colors = pl.cm.nipy_spectral(np.linspace(0,1,M))

ax[0].set_title(f'Modal ranks for CE2 w/ rand IC')
for i in np.arange(6):
    ax[0].plot(R_ce2_sf[:,i],label=i+1,c=colors[i+1])
    
ax[1].set_title(f'Modal ranks for CE2 w/ QL IC')
for i in np.arange(M-1):
    ax[1].plot(R_ce2_sf_qlic[:,i],label=i+1,c=colors[i+1])

ax[1].legend(loc=5,ncol=1)

for a in ax:
    a.set_xlabel(r'$t$',fontsize=14)
    a.set_ylabel(r'$Rank$',fontsize=14)

plt.show()
# plt.savefig(dn+'figures/ranks_ce2_pj.png',bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(12,6))

index = -1

mymin = min(ql_sf['Fyym3'][:,:,index].flatten())
mymax = max(ql_sf['Fyym3'][:,:,index].flatten())

ax[0].set_title(f'QL')
im = ax[0].imshow((ql_sf['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",vmin=mymin,vmax=mymax,norm=MidpointNormalize(mymin, mymax, 0.))
fig.colorbar(im)



mymin = min(ce2_sf['Fyym2'][:,:,index].flatten())
mymax = max(ce2_sf['Fyym2'][:,:,index].flatten())

ax[1].set_title(f'CE2')
im = ax[1].imshow((ce2_sf['Fyym2'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))
fig.colorbar(im)

ax[2].set_title(f'CE2 w/ QL IC')
im = ax[2].imshow((ce2_sf_qlic['Fyym1'][:,:,index]),cmap="RdBu_r",origin="lower",interpolation="none",norm=MidpointNormalize(mymin, mymax, 0.))

# fig.subplots_adjust(bottom=0.2)
# cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.04])
# fig.colorbar(im, cax=cbar_ax,orientation='horizontal')

plt.title('Second cumulant for m = 1 at t = {:0.0f} days'.format(ql_pj['t'][index]),fontsize=14)

for a in ax:
    a.set_ylabel(r'$y_2$',fontsize=14)
    a.set_xlabel(r'$y_1$',fontsize=14)
    a.set_xticks([])
    a.set_yticks([])

plt.show()